# Questions


# Objectives
* 'develop' a workflow
* query table names
* query column names
* write functions that are useful to you

In [1]:
import sqlite3
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
conn = sqlite3.Connection("./data/data.sqlite")

In [5]:
cur = conn.cursor()

In [6]:
# querying all table_names

def get_table_names(conn):
    query = """
            select name from sqlite_master where type='table';
            """
    res = conn.execute(query).fetchall()
    table_names = [r[0] for r in res]
    return table_names

In [7]:
get_table_names(conn)

['productlines',
 'offices',
 'customers',
 'employees',
 'orderdetails',
 'orders',
 'payments',
 'products']

In [14]:
# query all column names in a table
# I only use PRAGMA to get column information
def get_table_column_names(conn, table_name):
    query = f"""
            pragma table_info({table_name})
            """
    res = conn.execute(query).fetchall()
    col_names = [r[1] for r in res]
    return col_names

In [15]:
get_table_column_names(conn, 'offices')

['officeCode',
 'city',
 'phone',
 'addressLine1',
 'addressLine2',
 'state',
 'country',
 'postalCode',
 'territory']

# Connection vs Cursor
* Connection
    * makes calls to the db
    * db returns results to the connection 
    * usually a row limit 
* Cursor
    * make calls to the db
    * interact with the results of the return values
        * ex: pagination

In [25]:
res = cur.execute("select * from employees")
res.fetchone()

(1002,
 'Murphy',
 'Diane',
 'x5800',
 'dmurphy@classicmodelcars.com',
 1,
 '',
 'President')

In [37]:
# display function
def load_table_as_df(conn, table_name, to_display=True):
    query = f"""
            select * from {table_name}
            """
    df = pd.read_sql(query, conn)
    if to_display:
        display(df.head())
    return df


def display_table_head(conn, table_name):
    query = f"""
            select * from {table_name}
            """
    df = pd.read_sql(query, conn)
    display(df.head(2))
    return

In [38]:
display_table_head(conn, 'employees')
display_table_head(conn, 'offices')

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales


,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA


In [49]:
employees = load_table_as_df(conn, 'employees')
offices = load_table_as_df(conn, 'offices')

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)


,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan


In [47]:
# IN command for sql

# let's get employees with numbers 1002 - 1010 

nums = tuple(list(range(1002, 1011)))
query = f"""
        select * from employees
        where employeeNumber in {nums}
        """

df = pd.read_sql(query, conn)
df.head()

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President


# Subquery Stuff
## let's get all the employees in an office that is located in the USA

In [50]:
conn.execute("""select offices.officeCode from offices
            where country='USA'""").fetchall()

[(1,), (2,), (3,), (27,)]

In [53]:
query = """
        select * from employees 
        where employees.officeCode in 
            (select offices.officecode from offices
            where country='USA');"""

df = pd.read_sql(query, conn)
df.head()

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056,Sales Manager (NA)
4,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep


# what did we learn? 
* seeing a workflow/process
* can you call a subquery a nested query? Yes!
* display tables with code
* get table names and column names
* use functions to make life easier 
* sql is not case sensitive use join and display data as you go
* alias queries 